<a href="https://colab.research.google.com/github/Max-FM/SPRINT-Colombia/blob/main/Colombia_Farms_Time_Series_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Colombia Farms Time Series Data

**Disclaimer:** This notebook is partially adapted from [this](https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/time-series-visualization-with-altair/tutorial.ipynb) tutorial on visualising time-series data using Google Earth Engine and Python. This is a good reference to further understand how this notebook works.

**Note:** This notebook will not work straight 'out of the box' and will require a bit of editing to suit your purposes. The idea of this notebook is to show the method I used to obtain time-series data from Google Earth Engine and is intended to be used as a template for future work.

##Import required packages

In [1]:
import ee
import numpy as np
import pandas as pd

##Authenticate Google Earth Engine

To access the Google Earth Engine API you require an account. To request access, go to [https://signup.earthengine.google.com](https://signup.earthengine.google.com/). You may have to wait up to a day or so to be granted access and it's possible you will not recieve any email communication. To manually check whether you have access, try to log into [https://code.earthengine.google.com](https://code.earthengine.google.com/), or attempt to run the next cell and follow the instructions provided in the output cell.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=pYIJH4U7NI4Ays6wgTNXJfOfTg-6QlunEYb-w3DtTK0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4-WJFlOJDECSeBcExu1vkl4Ra2l0MXz68PmeuDtbUrZn6gYkPHARE

Successfully saved authorization token.


## Define Request Function

This function contains a series of image collections on Google Earth Engine that we wish to obtain time series data from.

In [5]:
def calculate_RVI(image):
    bandDict = {
                'VV': image.select('VV'), 
                'VH': image.select('VH')
                }

    rvi = image.expression('(4*VH)/(VH+VV)', bandDict) \
               .rename('RVI')

    return image.addBands(rvi)

def get_Sentinel_1_RVI(criteria):
    SENTINEL_1_SAR_GRD_C_BAND = ee.ImageCollection("COPERNICUS/S1_GRD") \
                                  .filter(criteria) \
                                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
                                  .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                                  .select(['VV', 'VH'])

    SENTINEL_1_RVI = SENTINEL_1_SAR_GRD_C_BAND.map(calculate_RVI).select('RVI')

    return SENTINEL_1_RVI

def obtain_data(region):
    criteria  = ee.Filter.geometry(region) 

    SENTINEL_1_RVI = get_Sentinel_1_RVI(criteria)

    MODIS_16D_TERRA_VEG = ee.ImageCollection("MODIS/006/MOD13Q1") \
                             .filter(criteria) \
                             .select(['NDVI', 'EVI'])

    MODIS_16D_AQUA_VEG = ee.ImageCollection("MODIS/006/MYD13Q1") \
                           .filter(criteria) \
                           .select(['NDVI', 'EVI'])

    TERRA_CLIMATE_MONTHLY = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE") \
                              .filter(criteria)

    PERSIANN_DAILY = ee.ImageCollection("NOAA/PERSIANN-CDR") \
                       .filter(criteria)
                       
    CHIRPS_DAILY = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                     .filter(criteria)

    IMERG_MONTHLY = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06") \
                      .filter(criteria)

    ERA5_DAILY = ee.ImageCollection("ECMWF/ERA5/DAILY") \
                   .filter(criteria)

    ERA5_MONTHLY = ee.ImageCollection("ECMWF/ERA5/MONTHLY") \
                     .filter(criteria)

    # Above image collections are placed in a Python dictionary for convenience.
    image_collections = {
                         'SENTINEL_1_RVI': SENTINEL_1_RVI,
                         'MODIS_16D_TERRA_VEG': MODIS_16D_TERRA_VEG,
                         'MODIS_16D_AQUA_VEG': MODIS_16D_AQUA_VEG,
                         'TERRA_CLIMATE_MONTHLY': TERRA_CLIMATE_MONTHLY,
                         'PERSIANN_DAILY': PERSIANN_DAILY,
                         'CHIRPS_DAILY': CHIRPS_DAILY,
                         'IMERG_MONTHLY': IMERG_MONTHLY,
                         'ERA5_DAILY': ERA5_DAILY,
                         'ERA5_MONTHLY': ERA5_MONTHLY
                         }

    return image_collections

## Define Region Reduction and Data Collection Functions


In [6]:
def create_reduce_region_function(
                                  geometry,
                                  reducer=ee.Reducer.median(),
                                  scale=250,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4
                                  ):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
                            reducer=reducer,
                            geometry=geometry,
                            scale=scale,
                            crs=crs,
                            bestEffort=bestEffort,
                            maxPixels=maxPixels,
                            tileScale=tileScale
                            )

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

# Define a function to transfer feature properties to a dictionary.
def feature_collection_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

# Aggregates pixel values over a defined region in a Google Earth Engine image 
# collection and returns a time-series Pandas dataframe.
def obtain_timeseries_dataframe(
                                image_collection, 
                                region, 
                                aggregation, 
                                scale=250,
                                crs='EPSG:4326',
                                bestEffort=True,
                                maxPixels=1e13,
                                tileScale=4
                                ):
    
    reduce_dict = {'median': ee.Reducer.median(),
                   'stdDev': ee.Reducer.stdDev(),
                   'count': ee.Reducer.count()}
    
    # Defines the reduction function to apply to the image collection.
    reduce_fn = create_reduce_region_function(geometry=region, 
                                              reducer=reduce_dict[aggregation], 
                                              scale=scale,
                                              crs=crs,
                                              bestEffort=bestEffort,
                                              maxPixels=maxPixels,
                                              tileScale=tileScale)
    
    # Creates a time-series feature collection by aggregating over the image 
    # collection.
    feature_collection_reduced = ee.FeatureCollection(image_collection.map(reduce_fn)) \
                                   .filter(ee.Filter.notNull(image_collection.first().bandNames()))

    # Converts the feature collection to a Python dictionary.
    reduced_dict = feature_collection_to_dict(feature_collection_reduced).getInfo()

    # Converts the Python dictionary to a Pandas dataframe.
    df = pd.DataFrame(reduced_dict)

    # Renames data columms to reflect aggregation type.
    df.columns = df.columns.map(lambda colName : colName+f'_{aggregation}' if colName !='system:index' and colName !='millis' else colName)
    # Drops the 'millis' column.
    df = add_date_info(df).drop(columns=['millis'])

    return df

# Filters an image collection to within a certain date range.
def filter_collection_by_date(collection, start_date, end_date):
    start_date = ee.Date(start_date)
    end_date = ee.Date (end_date)

    date_filter = ee.Filter.date(start_date, end_date)

    collection_filtered = collection.filter(date_filter)

    return collection_filtered

# Sorts the columns alphabetically with the exeption of a pre-detemined list of
# columns.
def reorder_dataframe(df, preordered_columns):
    df = df.reindex(sorted(df.columns), axis=1)
    new_order = preordered_columns + [c for c in df.columns if c not in preordered_columns]
    df = df.reindex(columns=new_order)
    return df

def add_info_columns(df, district_name, farm_name, collection_name):
    df['District'] = district_name
    df['Farm'] = farm_name
    df['Image_Collection'] = collection_name

    return df

## Testing the Code on a Single Farm

Have commented this section out for now. Feel free to uncomment and play around with if you wish to test the methods on a single farm.

In [7]:
# farm = ee.FeatureCollection(f'users/maxfoxley-marrable/SPRINT/Farms/Dosquebradas_Central_N_Farm_C')
# aoi = farm.geometry()

# start_date = '2019-01-01'
# end_date = '2020-01-01'

# image_collections = obtain_data(region=aoi)


# collection = image_collections['TERRA_CLIMATE_MONTHLY']
# collection = filter_collection_by_date(collection, start_date, end_date)

In [8]:
# %%time

# scale = 10 # If possible, downsample to pixel resolution in metres.

# median_df = obtain_timeseries_dataframe(collection,
#                                         region=aoi, 
#                                         aggregation='median', 
#                                         scale=scale)

# std_dev_df = obtain_timeseries_dataframe(collection,
#                                          region=aoi, 
#                                          aggregation='stdDev', 
#                                          scale=scale)

# count_df = obtain_timeseries_dataframe(collection,
#                                        region=aoi, 
#                                        aggregation='count', 
#                                        scale=scale)

In [9]:
# display(median_df.head(), median_df.info())
# display(std_dev_df.head())
# display(count_df.head())

# columns_of_interest = ['system:index', 'Timestamp', 'Year', 'Month', 'Day', 'DOY']

# merge = pd.merge_ordered(median_df, std_dev_df, on=columns_of_interest)
# merge = reorder_dataframe(merge, columns_of_interest)
# merge['Pixel_Count'] = count_df.iloc[:,0]

# merge
# # merge.sample(30)


##Getting Data for Multiple Farms

In [10]:
# Creates sub-folders for each farm.

# for farm in farms_list:
#     district_name = farm.partition('_')[0]
#     farm_name = farm.partition('_')[-1]
#     dir_name = f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/{district_name}/Farms/CSV Data/{farm_name}'
    
#     !mkdir '{dir_name}'

In [11]:
# List of feature collections that contain shapefiles for farms.
farms_list = [
              'Dosquebradas_Central_N_Farm_C', 
              'Dosquebradas_Dos_Cosechas_C',
              'Dosquebradas_Fincas_campesinas_C',
              'Dosquebradas_La_Union_C',
              'Dosquebradas_Las_Camillas_C',
              'Dosquebradas_Minas_del_Socorro_C',
              'Dosquebradas_NE_area_C',
              'Dosquebradas_N_Farm_C',
              'Dosquebradas_New_coffee_farm_C',
              'Dosquebradas_Santa_Ana_Alto_C',
              'Dosquebradas_Santa_Ana_Bajo_La_Union_C',
              'Versailles_Barcelona',
              'Versailles_El_Chalet',
              'Versailles_Guyabel',
              'Versailles_La_Primavera',
              'Versailles_La_Rivera',
              'Versailles_Miraflores',
              'Versailles_Predio_Valery',
              'Versailles_Roberto',
              'Versailles_San_Fernando',
              'Versailles_Villa_Emanuel'
              ]

# Splitting date range into yearly chunks to reduce timeouts on GEE side.
date_ranges = [(f'{year}-01-01', f'{year+1}-01-01') for year in np.arange(1997, 2021)]

# Resample pixels to resolution in metres. Setting native scale where known.
scaleDict = {
            'SENTINEL_1_RVI': 10,
            'MODIS_16D_TERRA_VEG': 50,
            'MODIS_16D_AQUA_VEG': 50, 
            'TERRA_CLIMATE_MONTHLY': 50,
            'PERSIANN_DAILY': 50,
            'CHIRPS_DAILY': 50,
            'IMERG_MONTHLY': 50,
            'ERA5_DAILY': 50,
            'ERA5_MONTHLY': 50
            }

# List of pre-ordered columns to prefix the dataframe with.
preordered_columns = ['District', 'Farm', 'Year', 'Month', 'Day', 'DOY', 'Timestamp', 'Image_Collection', 'system:index']
# List of columns to merge the median and standard deviation dataframes on.
merge_columns = ['system:index', 'Timestamp', 'Year', 'Month', 'Day', 'DOY']

# Set to 'None' to iterate through all image collections defined above.
# Otherwise provide a list of strings 
# e.g. ['MODIS_16D_TERRA_VEG', 'PERSIANN_DAILY', etc]
desired_collections = None

# Set to 'True' to overwrite existing files.
overwrite = False

In [ ]:
%%time
import os.path

# Loops through the shapefiles for each farm.
for farm in farms_list:
    # Shape files have been pre-uploaded to Google Earth Engine.
    region = ee.FeatureCollection(f'users/maxfoxley-marrable/SPRINT/Farms/{farm}')
    aoi = region.geometry()

    # Splits the filename string to obtain the district and farm name.
    district_name = farm.partition('_')[0]
    farm_name = farm.partition('_')[-1]

    print(district_name, farm_name)

    # Obtains all image collections defined above for said region.
    image_collections = obtain_data(region)
    
    # Filters out unwanted image collections.
    if desired_collections:
        image_collections = {collection: image_collections[collection] \
                            for collection in desired_collections}

    # Loops for each image collection obtained above.
    for collection_name, collection in image_collections.items():
        
        print(collection_name)

        # Edit as required. For now I have it directly saving to the shared 
        # Google Drive.
        filepath = f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/{district_name}/Farms/CSV Data/{farm_name}/{farm_name}_{collection_name}.csv'

        # Skips if CSV file for a specific region and image collection already 
        # exists, provided overwrite = False.
        if overwrite == False and os.path.isfile(filepath) == True:
            print(f'{farm_name}_{collection_name}.csv already exists, skipping.')
            continue

        # Creates an empty master dataframe which will be iteratively appended 
        # to create a full ~20 year timeseries. 
        dataframe = pd.DataFrame()

        # Skips the image collection if it contains no images.
        if collection.size().getInfo() == 0:
            continue
        
        # Loops though dates in batches of about 8 years to avoid timeouts on
        # GEE server side. See above. 
        for start_date, end_date in date_ranges:
            date_range_str = f'{start_date}_{end_date}'

            # print(date_range_str)

            # Filters image collection to within desired date range.
            collection_filtered = filter_collection_by_date(collection, start_date, end_date)

            if collection_filtered.size().getInfo() == 0:
                continue

            # Gets the median pixel values for a region across all images in
            # the filtered image collection.
            median_df = obtain_timeseries_dataframe(collection_filtered,
                                                    region=aoi, 
                                                    aggregation='median', 
                                                    scale=scaleDict[collection_name])
            
            # Same as above, but gets the standard deviation.
            std_dev_df = obtain_timeseries_dataframe(collection_filtered,
                                                     region=aoi, 
                                                     aggregation='stdDev', 
                                                     scale=scaleDict[collection_name])
            
            # Gets the pixel count for the region, but there has to be a better 
            # way of pixel counting.
            count_df = obtain_timeseries_dataframe(collection_filtered,
                                                   region=aoi, 
                                                   aggregation='count', 
                                                   scale=scaleDict[collection_name])
            

            # Merges median and standard deviation time-series data frames into
            # a single dataframe.
            merge_df = pd.merge_ordered(median_df, std_dev_df, on=merge_columns)
            merge_df = add_info_columns(merge_df, district_name, farm_name, collection_name)
            merge_df = reorder_dataframe(merge_df, preordered_columns)
            merge_df['Pixel_Scale'] = scaleDict[collection_name] # Pixel scale (in metres) defined above.
            merge_df['Pixel_Count'] = count_df.iloc[:,0] # Pixel count, this might be a bad way of doing this.
                
            # Appends merged dataframe for a master dataframe containing the 
            # full ~20 year time-series.
            dataframe = dataframe.append(merge_df, ignore_index=True)

        # Saves master dataframe to a CSV file.
        dataframe.to_csv(filepath)

Versailles Villa_Emanuel
SENTINEL_1_RVI
MODIS_16D_TERRA_VEG
MODIS_16D_AQUA_VEG
TERRA_CLIMATE_MONTHLY
PERSIANN_DAILY
CHIRPS_DAILY
IMERG_MONTHLY
ERA5_DAILY
ERA5_MONTHLY
